# 1. Librerias y módulos

### 1.1. Instalar librerias

In [ ]:
# Extracción de datos
!pip install pdfminer.six
!pip install pdfquery
!pip install 'PyPDF2<3.0'
!pip install "camelot-py[base]"
!pip install ghostscript
!apt install ghostscript python3-tk
!pip install xlsxwriter

# Conversión a jpg
!pip install pdf2image
!sudo apt-get install python-poppler
!sudo apt-get install poppler-utils

# Conversion a txt
!sudo add-apt-repository ppa:alex-p/tesseract-ocr-devel -y
!sudo apt-get update
!sudo apt-get install -y tesseract-ocr
!sudo apt-get install tesseract-ocr-spa
!pip install pytesseract
!pip install pyyaml==5.1
!pip install unidecode

### 1.2. Importar librerias

In [ ]:
# Google Colab y Drive
from google.colab import drive
import sys

# Extracción de datos
import os
import io
import camelot
import ghostscript
import PyPDF2
import re
from io import StringIO
import pandas as pd
import numpy as np
from pdfquery import PDFQuery
from pathlib import Path
import itertools
from itertools import chain
import matplotlib.pyplot as plt
from matplotlib import patches
import pdfminer
from pdfminer.high_level import extract_pages
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfdevice import PDFDevice
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.layout import LTPage
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.layout import LTTextBoxVertical
from pdfminer.layout import LTTextLineHorizontal
from pdfminer.layout import LTTextLineVertical
from pdfminer.layout import LTTextContainer
from pdfminer.layout import LTChar
from pdfminer.layout import LTText
from pdfminer.layout import LTTextBox
from pdfminer.layout import LTAnno

# Conversion a jpg
from pdf2image import convert_from_path
from pathlib import Path
import PIL.Image
from glob import glob

# Conversion a txt
import json
import cv2
import random
import cv2
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
from pdf2image import pdfinfo_from_path

### 1.3. Importar el módulo "funciones"

In [ ]:
drive.mount('/content/gdrive')
!cat '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/funciones.py'
sys.path.append('/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento')

In [ ]:
import funciones as fun

# 2. Modalidad: Auditoría de Cumplimiento

In [ ]:
modalidad           = 'AC'
modalidad_ext       = 'AUDITORIA_CUMPLIMIENTO'

## 2.1. Extraer archivos resumen

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/datos/SERVICIO_CONTROL_POSTERIOR/{ modalidad_ext }/*/*-resumen.pdf' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/1_archivos_resumen'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'

txt_file_path = os.path.join( txt_output_path, f'{ modalidad }_resumen_info.txt.txt' )

with open( txt_file_path, 'w+' ) as file:
    for i, input_element in enumerate( input_path ):

        try:
            input_element_name                    = input_element[ input_element.rindex( '/' ) + 1: ]
            input_element_name                    = os.path.splitext( input_element_name )[ 0 ]

            extracted_input_element               = fun.extract_all( input_element, modalidad )
            extracted_input_element[ 'doc_name' ] = input_element_name

            output_element_path = os.path.join( output_path, f'{ input_element_name }.xlsx' )

            extracted_input_element.to_excel( output_element_path )

            line = f'Success extracting: { output_element_path }\t{ input_element }\n'
            file.write( line )

        except Exception as e:
            error_line = f'Error extracting: { input_element }\n'
            file.write( error_line )

## 2.2. Hacer append de archivos resumen

In [ ]:
input_path          = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/1_archivos_resumen/*.xlsx'
output_path         = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados'

input_elements      = glob( input_path )
tables              = []

for input_element in input_elements:

    table           = pd.read_excel( input_element, dtype = { 'dni': object } )
    table           = table.fillna( '.' )

    tables.append( table )

ac_summary_table    = pd.concat( tables, axis = 0 )
output_element_path = os.path.join( output_path, f'{ modalidad }_summary_table.xlsx' )

ac_summary_table.to_excel( output_element_path )

## 2.3. Extraer fechas

### 2.3.1. Convertir la primera página de los informes a formato jpg

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/datos/SERVICIO_CONTROL_POSTERIOR/{ modalidad_ext }/*/*-informe.pdf' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/2_archivos_jpg'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'

fun.convert_1p_to_jpg( input_path, output_path, txt_output_path, modalidad )

### 2.3.2. Convertir los archivos jpg a formato txt usando OCR

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/2_archivos_jpg/*-informe.jpg' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/3_archivos_txt'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'

fun.convert_1p_to_text( input_path, output_path, txt_output_path, modalidad )

## 2.3.3. Extraer las fechas usando regex

In [ ]:
input_path      = glob( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/3_archivos_txt/*-informe.txt' )
output_path     = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/4_archivos_dates_xlsx'
txt_output_path = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/6_archivos_registro'
regex           = r'(?P<day1>\d{1,2})\s?(De|de|DE|DE-|E|DE.)?(\:|-)?\s?(?P<month1>\w{4,10})?\s?(\—)?\s?(De|DE|de|E|e|ce|Es|DE-|DE/|DE.)?\s?(-)?\s?(?P<year1>\d{4})?\s?(Al|aL|AL|A|al|a|¿AL)\s?(?P<day2>\d{1,2})\s?(De|de|DE|DE-|E|DE.)?\s?(BS)?\s?(?P<month2>\w{4,10})\n?(.)?(/)?\s?(De|DE|de|E|e|ce|Es|DE-|DE/|DE.)?\n?(OS|nd|-)?\s?(?P<year2>\d{4})'

fun.extract_dates( input_path, output_path, txt_output_path, modalidad, regex )

## 2.3.4. Hacer un append vertical de las fechas extraidas

In [ ]:
input_path   = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/4_archivos_dates_xlsx/*.xlsx'
output_path  = f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados'

fun.append_dates( input_path, output_path, modalidad )

,numero_informe,inicio,final
0,008-2018-2-3613,01/09/2015,31/12/2016
1,008-2018-2-5323,01/01/2016,31/12/2016
2,008-2019-2-2168,28/11/2018,31/12/2018
3,008-2019-2-2057,13/02/2017,27/09/2017
4,008-2019-2-0328,23/03/2018,02/07/2018
...,...,...,...
1072,008-2018-2-0446,01/07/2015,29/10/2015
1073,008-2018-2-2176,01/01/2016,31/12/2017
1074,008-2018-2-0453,16/01/2016,11/01/2018
1075,008-2018-2-1326,02/01/2015,31/12/2016


# 2.4. Merge

In [ ]:
ac_summary_table                     = pd.read_excel( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados/{ modalidad }_summary_table.xlsx' )
ac_dates                             = pd.read_excel( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/5_archivos_concatenados/{ modalidad }_fechas.xlsx' )

ac_summary_table[ 'numero_informe' ] = ac_summary_table[ 'numero_informe' ].astype( str ).str.strip().\
                                                                                          str.replace( '/', '_' ).\
                                                                                          str.replace( '\\', '_', regex = True )
ac_dates[ 'numero_informe' ]         = ac_dates[ 'numero_informe' ].astype( str ).str.strip()

ac_total = pd.merge( ac_summary_table,
                     ac_dates,
                     how = 'outer',
                     on  = 'numero_informe' )

ac_total.to_excel( f'/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/{ modalidad }/7_archivos_finales/{ modalidad }_total.xlsx' )

# 2.5. Append

In [ ]:
ac_total          = pd.read_excel( '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/AC/7_archivos_finales/AC_total.xlsx' )
sehpi_total       = pd.read_excel( '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/SEHPI/7_archivos_finales/SEHPI_total.xlsx' )

contraloria_final = pd.concat( [ ac_total, sehpi_total ], axis = 0 )

contraloria_final = contraloria_final.drop( ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'nombres' ], axis = 1 )
contraloria_final = contraloria_final.reindex( columns = [ 'numero_informe', 'titulo_informe/asunto', 'objetivo', 'entidad_auditada',
                                                           'monto_auditado', 'monto_examinado', 'monto_objeto_servicio', 'fecha_emision_informe',
                                                           'unidad_emite_informe', 'distrito', 'provincia', 'region', 'observaciones',
                                                           'recomendaciones', 'argumentos_de_hecho', 'modalidad', 'dni', 'personas', 'columnas',
                                                           'civil', 'penal', 'admin', 'adm_ent', 'adm_pas', 'doc_name', 'inicio', 'final'
                                                             ] )

contraloria_final.to_excel( '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/base_final/contraloria_final.xlsx' )

SEHPI:
* XLSX: 2023
* JPG:  2022
* TXT:  1913

AC:
* XLSX: 1148
* JPG:  1148
* TXT:  1066
* TXT2: 1077

regex           = r'(?P<day1>\d{1,2})\s?(de|DE|E)?(\:|-)?\s?(?P<month1>\w{4,10})?\s?(\—)?\s?(DE|de|AL|DEL)?\s?(-)?\s?(?P<year1>\d{4})?\s?(AL|A|al|a|¿AL)\s?(?P<day2>\d{1,2})\s?(DE|de)?\s?(?P<month2>\w{4,10})\n?(.)?\s?(DE|de|E|e|ce|Es|DE-|DE/)?\n?\s?(?P<year2>\d{4})'


***FIN***

In [ ]:
base_test = contraloria_final.copy()
base_test[ 'doc_name_2' ] = base_test[ 'doc_name' ].str.replace( '-resumen', '' )
base_test_filtered = base_test[base_test['numero_informe'] != base_test['doc_name_2']]
base_test_filtered.to_excel( '/content/gdrive/MyDrive/DATOS_CONTRALORÍA/procesamiento/test/base_test_filtered.xlsx' )